# Import Modules

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Add experiment date here to apply to dataframe
analysis_date = '20210903'

path = os.path.abspath('')+'/csvs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe


for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label
    df['Fluor'], df['ROI'] = zip(*df['Label'].map(lambda x: x.split(':')))
    # Split values in Image name column
    (df['ExptDate'], df['Treatment'], df['Dose'], df['Stains'], df['Embryo'], 
        df['Somites'], df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,,Label,Area,Mean,IntDen,RawIntDen,Image,Fluor,ROI,ExptDate,Treatment,Dose,Stains,Embryo,Somites,Section
0,1,H2BRFP:background,73.274,8.737,640.196,12424.0,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;BREH2B...,H2BRFP,background,20210824,SMPD3FLAG,3ugul,BF;Pax7;H2BRFP;BREH2Bd2EGFP,Emb1,6ss,sec1
1,2,H2BRFP:background,50.395,8.586,432.689,8397.0,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;BREH2B...,H2BRFP,background,20210824,SMPD3FLAG,3ugul,BF;Pax7;H2BRFP;BREH2Bd2EGFP,Emb1,6ss,sec1
2,3,H2BRFP:background,169.118,8.615,1456.879,28273.0,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;BREH2B...,H2BRFP,background,20210824,SMPD3FLAG,3ugul,BF;Pax7;H2BRFP;BREH2Bd2EGFP,Emb1,6ss,sec1
3,4,H2BRFP:Cntl,5423.015,181.029,981721.130,19051818.0,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;BREH2B...,H2BRFP,Cntl,20210824,SMPD3FLAG,3ugul,BF;Pax7;H2BRFP;BREH2Bd2EGFP,Emb1,6ss,sec1
4,5,H2BRFP:Expt,4504.665,102.197,460364.517,8934086.0,20210824_SMPD3FLAG_3ugul_BF;Pax7;H2BRFP;BREH2B...,H2BRFP,Expt,20210824,SMPD3FLAG,3ugul,BF;Pax7;H2BRFP;BREH2Bd2EGFP,Emb1,6ss,sec1


In [5]:
# Get a list of treatments and dates
treatment_list = full_df.Treatment.unique().tolist()
date_list = full_df.ExptDate.unique().tolist()

control_treat = 'H2B-RFP'
experiment_treat = 'nSMase2-FLAG'

# Parse out specific somite stages
data = full_df.loc[full_df['Somites'].isin(['8ss', '9ss'])]
# data = full_df.loc[full_df['Somites'].isin(['7ss','6ss'])]

# Mean background values and group by Treatment, Embryo, Fluor, ROI and Section
mean_sections = ((data.groupby(['ExptDate', 'Treatment', 'Dose', 'Embryo', 'Somites', 'Fluor', 'ROI', 'Section'])
                  ['Area', 'Mean', 'IntDen']).mean())

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
full_results = pd.DataFrame()
list_ = []

for j in date_list:
    date = j
    df_date = pd.DataFrame(mean_sections.xs(date))
    
    for i in treatment_list:
        # Slice dataframe to process only embryos with given treatment
        treatment = i
        df_treatment = pd.DataFrame(df_date.xs(treatment))

        # Determine CTCF values = ROI IntDen - (background mean * ROI area)
        # Calculate background (background mean * ROI area)
        background_corr_cntl = (df_treatment.xs('background', level='ROI')['Mean'] 
                        * df_treatment.xs('Cntl', level='ROI')['Area'])
        background_corr_expt = (df_treatment.xs('background', level='ROI')['Mean'] 
                        * df_treatment.xs('Expt', level='ROI')['Area'])

        # Slice out only Cntl or Expt values in IntDen
        intdens_cntl = df_treatment.xs('Cntl', level='ROI')['IntDen'] 
        intdens_expt = df_treatment.xs('Expt', level='ROI')['IntDen'] 

        # Subtract background from IntDens to determine CTCF and concatenate into single dataframe
        sub_cntl = pd.DataFrame(intdens_cntl - background_corr_cntl)
        sub_expt = pd.DataFrame(intdens_expt - background_corr_expt)
        full_ctcf = pd.concat([sub_cntl, sub_expt], keys = ['Cntl', 'Expt'])
        full_ctcf.columns = ['CTCF']

        # Pull out TCFLef and H2BRFP values
        ctcf_tcflef = full_ctcf.xs('BREH2Bd2EGFP', level='Fluor')['CTCF'] 
        ctcf_RFP = full_ctcf.xs('H2BRFP', level='Fluor')['CTCF'] 

        # Normalize for electroporation efficiency by determining TCFLef/RFP
        electroporation_norm = pd.DataFrame(ctcf_tcflef / ctcf_RFP)
        electroporation_norm.columns = ['CTCF']
        electroporation_norm.index.names = ['Side', 'Dose', 'Embryo', 'Somites', 'Section']

        # Average sections grouped by embryos before generating Expt/Cntl ratio
        averaged_sections = electroporation_norm.groupby(['Side','Embryo', 'Somites']).mean()

        # Pull out Cntl and Expt CTCFs
        ctcf_cntl = averaged_sections.xs('Cntl', level='Side')['CTCF'] 
        ctcf_expt = averaged_sections.xs('Expt', level='Side')['CTCF'] 

        # Generate ratios as Expt/Cntl
        ratios_sections = pd.DataFrame(ctcf_expt / ctcf_cntl)
        ratios_sections.columns = ['Expt/Cntl CTCF']

        # Normalize individual values to mean of control group
        norm_cntl = ctcf_cntl/(float(ctcf_cntl.mean()))
        norm_cntl = pd.DataFrame(norm_cntl)
        norm_cntl.columns = [control_treat]
        norm_expt = ctcf_expt/(float(ctcf_cntl.mean()))
        norm_expt = pd.DataFrame(norm_expt)
        norm_expt.columns = [experiment_treat]

        # Combine processed values into single dataframe and output as csv file 'Results.csv'
        ctcf_cntl = pd.DataFrame(ctcf_cntl)
        ctcf_cntl.columns = ['Cntl CTCF']
        ctcf_expt = pd.DataFrame(ctcf_expt)
        ctcf_expt.columns = ['Expt CTCF']
        results = (pd.concat([ctcf_cntl, ctcf_expt, ratios_sections, norm_cntl, norm_expt], axis=1, sort=True)).reset_index()
        list_.append(results)
        
full_results = pd.concat(list_)
full_results.to_csv(analysis_date + '_' + treatment + '_CTCFResults.csv')

full_results

,Embryo,Somites,Cntl CTCF,Expt CTCF,Expt/Cntl CTCF,H2B-RFP,nSMase2-FLAG
0,Emb2,8ss,0.607785,0.406907,0.669491,0.774387,0.518445
1,Emb3,9ss,1.093073,0.498181,0.455762,1.392699,0.634739
2,Emb4,9ss,1.178489,1.200998,1.019099,1.501529,1.530207
3,Emb5,8ss,0.749621,0.838598,1.118695,0.955102,1.068468
4,Emb6,8ss,0.295330,0.197684,0.669369,0.376283,0.251872
0,Emb1,9ss,2.160728,2.506419,1.159988,1.262947,1.465004
1,Emb3,8ss,1.260995,0.882550,0.699884,0.737053,0.515851
